In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from scipy.stats import zscore

# Load the dataset
df = pd.read_csv('Spotify_Youtube.csv')

In [2]:
# Top rows of the dataset 
df.head()

,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,Loudness,...,Url_youtube,Title,Channel,Views,Likes,Comments,Description,Licensed,official_video,Stream
0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,-6.679,...,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),Gorillaz,693555221.0,6220896.0,169907.0,Official HD Video for Gorillaz' fantastic trac...,True,True,1.040235e+09
1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Rhinestone Eyes,Plastic Beach,album,spotify:track:1foMv2HQwfQ2vntFf9HFeG,0.676,0.703,8.0,-5.815,...,https://www.youtube.com/watch?v=yYDmaexVHic,Gorillaz - Rhinestone Eyes [Storyboard Film] (...,Gorillaz,72011645.0,1079128.0,31003.0,The official video for Gorillaz - Rhinestone E...,True,True,3.100837e+08
2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,spotify:track:64dLd6rVqDLtkXFYrEUHIU,0.695,0.923,1.0,-3.930,...,https://www.youtube.com/watch?v=qJa-VFwPpYA,Gorillaz - New Gold ft. Tame Impala & Bootie B...,Gorillaz,8435055.0,282142.0,7399.0,Gorillaz - New Gold ft. Tame Impala & Bootie B...,True,True,6.306347e+07
3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,On Melancholy Hill,Plastic Beach,album,spotify:track:0q6LuUqGLUiCPP1cbdwFs3,0.689,0.739,2.0,-5.810,...,https://www.youtube.com/watch?v=04mfKJWDSzI,Gorillaz - On Melancholy Hill (Official Video),Gorillaz,211754952.0,1788577.0,55229.0,Follow Gorillaz online:\nhttp://gorillaz.com \...,True,True,4.346636e+08
4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Clint Eastwood,Gorillaz,album,spotify:track:7yMiX7n9SBvadzox8T5jzT,0.663,0.694,10.0,-8.627,...,https://www.youtube.com/watch?v=1V_xRb0x9aw,Gorillaz - Clint Eastwood (Official Video),Gorillaz,618480958.0,6197318.0,155930.0,The official music video for Gorillaz - Clint ...,True,True,6.172597e+08


In [3]:
# Data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20718 entries, 0 to 20717
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Artist            20718 non-null  object 
 1   Url_spotify       20718 non-null  object 
 2   Track             20718 non-null  object 
 3   Album             20718 non-null  object 
 4   Album_type        20718 non-null  object 
 5   Uri               20718 non-null  object 
 6   Danceability      20716 non-null  float64
 7   Energy            20716 non-null  float64
 8   Key               20716 non-null  float64
 9   Loudness          20716 non-null  float64
 10  Speechiness       20716 non-null  float64
 11  Acousticness      20716 non-null  float64
 12  Instrumentalness  20716 non-null  float64
 13  Liveness          20716 non-null  float64
 14  Valence           20716 non-null  float64
 15  Tempo             20716 non-null  float64
 16  Duration_ms       20716 non-null  float6

In [4]:
print(df.dtypes)

Artist               object
Url_spotify          object
Track                object
Album                object
Album_type           object
Uri                  object
Danceability        float64
Energy              float64
Key                 float64
Loudness            float64
Speechiness         float64
Acousticness        float64
Instrumentalness    float64
Liveness            float64
Valence             float64
Tempo               float64
Duration_ms         float64
Url_youtube          object
Title                object
Channel              object
Views               float64
Likes               float64
Comments            float64
Description          object
Licensed             object
official_video       object
Stream              float64
dtype: object


In [5]:
print(df.describe())

       Danceability        Energy           Key      Loudness   Speechiness  \
count  20716.000000  20716.000000  20716.000000  20716.000000  20716.000000   
mean       0.619777      0.635250      5.300348     -7.671680      0.096456   
std        0.165272      0.214147      3.576449      4.632749      0.111960   
min        0.000000      0.000020      0.000000    -46.251000      0.000000   
25%        0.518000      0.507000      2.000000     -8.858000      0.035700   
50%        0.637000      0.666000      5.000000     -6.536000      0.050500   
75%        0.740250      0.798000      8.000000     -4.931000      0.103000   
max        0.975000      1.000000     11.000000      0.920000      0.964000   

       Acousticness  Instrumentalness      Liveness       Valence  \
count  20716.000000      20716.000000  20716.000000  20716.000000   
mean       0.291535          0.055962      0.193521      0.529853   
std        0.286299          0.193262      0.168531      0.245441   
min        0

In [6]:
# Handling missing values(fill with mean)
df.fillna(df.mean(numeric_only=True), inplace=True)

#Check number of null values
df.isnull().sum()

Artist                0
Url_spotify           0
Track                 0
Album                 0
Album_type            0
Uri                   0
Danceability          0
Energy                0
Key                   0
Loudness              0
Speechiness           0
Acousticness          0
Instrumentalness      0
Liveness              0
Valence               0
Tempo                 0
Duration_ms           0
Url_youtube         470
Title               470
Channel             470
Views                 0
Likes                 0
Comments              0
Description         876
Licensed            470
official_video      470
Stream                0
dtype: int64

In [7]:
from scipy.stats import zscore

# Identify outliers in numerical columns using z-score
# Selecting features for outlier detection
numerical_cols = ['Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms', 'Views', 'Likes', 'Comments', 'Stream']
z_scores = zscore(df[numerical_cols])
outliers = (abs(z_scores) > 3).any(axis=1)

# Display outlier rows
outlier_rows = df[outliers]
print(outlier_rows)

                      Artist  \
0                   Gorillaz   
4                   Gorillaz   
10     Red Hot Chili Peppers   
11     Red Hot Chili Peppers   
12     Red Hot Chili Peppers   
...                      ...   
20654         Sleepy Penguin   
20683              Jung Kook   
20685              Jung Kook   
20688              Jung Kook   
20717            SICK LEGEND   

                                             Url_spotify  \
0      https://open.spotify.com/artist/3AA28KZvwAUcZu...   
4      https://open.spotify.com/artist/3AA28KZvwAUcZu...   
10     https://open.spotify.com/artist/0L8ExT028jH3dd...   
11     https://open.spotify.com/artist/0L8ExT028jH3dd...   
12     https://open.spotify.com/artist/0L8ExT028jH3dd...   
...                                                  ...   
20654  https://open.spotify.com/artist/32Rb6mS5HQKCu8...   
20683  https://open.spotify.com/artist/6HaGTQPmzraVma...   
20685  https://open.spotify.com/artist/6HaGTQPmzraVma...   
20688  https://

In [8]:
# Remove outlier rows
df.drop(df[outliers].index, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17894 entries, 1 to 20716
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Artist            17894 non-null  object 
 1   Url_spotify       17894 non-null  object 
 2   Track             17894 non-null  object 
 3   Album             17894 non-null  object 
 4   Album_type        17894 non-null  object 
 5   Uri               17894 non-null  object 
 6   Danceability      17894 non-null  float64
 7   Energy            17894 non-null  float64
 8   Key               17894 non-null  float64
 9   Loudness          17894 non-null  float64
 10  Speechiness       17894 non-null  float64
 11  Acousticness      17894 non-null  float64
 12  Instrumentalness  17894 non-null  float64
 13  Liveness          17894 non-null  float64
 14  Valence           17894 non-null  float64
 15  Tempo             17894 non-null  float64
 16  Duration_ms       17894 non-null  float64
 17

In [9]:
# Encoding categorical variables if necessary
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# Feature Scaling
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.drop('Views', axis=1))

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(scaled_features, df['Views'], test_size=0.2, random_state=42)

# Initialize the regressors
lgbm = LGBMRegressor()
xgb = XGBRegressor()
catboost = CatBoostRegressor(verbose=0)  # 'verbose=0' to prevent a lot of output

# Fit the models
lgbm.fit(X_train, y_train)
xgb.fit(X_train, y_train)
catboost.fit(X_train, y_train)

# Make predictions
predictions_lgbm = lgbm.predict(X_test)
predictions_xgb = xgb.predict(X_test)
predictions_catboost = catboost.predict(X_test)

# Evaluate the models
mae_lgbm = mean_absolute_error(y_test, predictions_lgbm)
mse_lgbm = mean_squared_error(y_test, predictions_lgbm)
r2_lgbm = r2_score(y_test, predictions_lgbm)

mae_xgb = mean_absolute_error(y_test, predictions_xgb)
mse_xgb = mean_squared_error(y_test, predictions_xgb)
r2_xgb = r2_score(y_test, predictions_xgb)

mae_catboost = mean_absolute_error(y_test, predictions_catboost)
mse_catboost = mean_squared_error(y_test, predictions_catboost)
r2_catboost = r2_score(y_test, predictions_catboost)

print('\n\n')
print('LightGBM:')
print(f'MAE: {mae_lgbm}')
print(f'MSE: {mse_lgbm}')
print(f'R-squared: {r2_lgbm}')
print('------')
print('XGBoost:')
print(f'MAE: {mae_xgb}')
print(f'MSE: {mse_xgb}')
print(f'R-squared: {r2_xgb}')
print('------')
print('CatBoost:')
print(f'MAE: {mae_catboost}')
print(f'MSE: {mse_catboost}')
print(f'R-squared: {r2_catboost}')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000662 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5636
[LightGBM] [Info] Number of data points in the train set: 14315, number of used features: 26
[LightGBM] [Info] Start training from score 62921394.270276



LightGBM:
MAE: 19723008.087399136
MSE: 1965521542937128.8
R-squared: 0.8417908088501447
------
XGBoost:
MAE: 19444089.168643113
MSE: 2010519222354387.0
R-squared: 0.8381688457687395
------
CatBoost:
MAE: 19086667.62553945
MSE: 1810574321846418.0
R-squared: 0.8542628545561648
